# Manga PDF Downloader
## 01 - Introduction
There are free options to read manga online, but in some of them, we aren't able to download a PDF version of it to read it later. This is the case of manganelo.tv (https://ww5.manganelo.tv/), where we can just read manga in the page from images loading in the web page.

This is an example of links for diferent mangas:
* "*Beyond The Sky*": https://ww5.manganelo.tv/manga/manga-ne990439.<br>
* "*My Wife Is A Fox Spirit*": https://ww5.manganelo.tv/manga/manga-hr984452.<br>
* "*Fight Class 3*": https://ww5.manganelo.tv/manga/manga-cd980038.<br>

In this URL structure, we are able to notice that "*ne990439*", "*hr984452*" & "*cd980038*" represents the manga's unique codes.

Inside every manga's page, we have acces to the list of chapters and their information, as we can see in the next screeshot from the "Beyond The Sky"'s page: <br><br>
![Manga SS](./imgs/manga_ss.PNG)

And, inside a chapter's page we can see the images: <br><br>
![Chapter SS](./imgs/chapter_ss.PNG)

## 02 - Objective
* Build a program to download an specific number of chapters for an asked manga.
  * The program will asks for the unique code that appears in the url's manga.
  * It has to shows the list of chapters to ask for the searched chapters.
  * For the UI, we will work on streamlit app
  
## 03 - Building the program's structure
First, we import the needed libraries and create basic needed functions

In [1]:
import requests
import shutil
import os
from PIL import Image
from bs4 import BeautifulSoup

def int_to_str(num:int, num_digits:int):
    """This function return a string number from an integer number
    and a specific number of digits. For example:
    >>> int_to_str(12,4)
    '0012'
    >>> int_to_str(1,5)
    '00001'"""
    if type(num) != int:
        raise TypeError("num has to be an integer")
    elif type(num_digits) != int:
        raise TypeError("num_digits has to be an integer")
    elif len(str(num)) > num_digits:
        raise ValueError("num_digits has to be bigger or equal than the number of digits of num")
    elif len(str(num)) == num_digits:
        return str(num)
    else:
        return '0'*(num_digits-len(str(num))) + str(num)

Before getting the images, we ask for the manga and chapters:

In [2]:
# Get the Manga URL
main_page = 'https://ww5.manganelo.tv/'
manga_code = 'ne990439' # Manga code example
manga_url = main_page + 'manga/manga-'+manga_code

# Get the chapters
manga_r = requests.get(manga_url)
if manga_r.status_code == 200: # If our requests was succesful
    manga_html = BeautifulSoup(manga_r.content, 'html.parser')
    chapters_elements = manga_html.find_all('a', class_='chapter-name text-nowrap')
    chapters_names = [element.text for element in chapters_elements]
    chapters_names.reverse() # List of chapers names ordered
    selected_chapters_names = ['Chapter 17', 'Chapter 24','Chapter 27','Chapter 30'] #Example of selected chapters
    # When the code moves to streamlit, this will be given by the user.
    selected_chapters_elements = [element for element in chapters_elements if element.text in selected_chapters_names]
else:
    print("Error when connecting to manga page.")
    selected_chapters_elements = []

Get the images downloaded in '*temp_imgs*', convert them to PDF and clean '*temp_imgs*'

In [3]:
# The 'a' HTML elements contains the URL for the specific character
for chapter_element in selected_chapters_elements:
    chapter_url = main_page + chapter_element['href']
    chapter_r = requests.get(chapter_url) #Get into chapter's page
    if chapter_r.status_code == 200:
        chapter_html = BeautifulSoup(chapter_r.content, 'html.parser')
        
        # Cleaning 'temp_imgs'
        try:
            shutil.rmtree('./temp_imgs')
            os.mkdir('./temp_imgs/')
        except:
            continue

        #Get all images in chapter
        imgs_elements = chapter_html.find_all('img', class_='img-loading') 
        num_pages = len(imgs_elements)
        num_digits = len(str(num_pages))
        counter = 0
        imgs_paths = []
        for img_element in imgs_elements: # For each img element
            img_url = img_element['data-src'] # Get url 
            img_ext = img_url.split('.')[-1]
            img_r = requests.get(img_url) 
            if img_r.status_code == 200:
                img_content = img_r.content # Get img
                path = './temp_imgs/'+int_to_str(counter, num_digits)+'.'+img_ext
                imgs_paths.append(path)
                with open(path, 'wb') as handler:
                    handler.write(img_content)
                counter += 1

        # Creating PDF
        imgs_list = [Image.open(path).convert('RGB') for path in imgs_paths[1:]]
        pdf_path = './mangas/'+manga_code+'-'+chapter_element.text+'.pdf'
        if os.path.isfile(pdf_path):
            os.remove(pdf_path)
        Image.open(imgs_paths[0]).convert('RGB').save(pdf_path, save_all=True, append_images=imgs_list)

        # Cleaning 'temp_imgs'
        try:
            shutil.rmtree('./temp_imgs')
            os.mkdir('./temp_imgs/')
        except:
            continue

    print('Chapter "'+chapter_element.text+'" saved.')

Chapter "Chapter 30" saved.
Chapter "Chapter 27" saved.
Chapter "Chapter 24" saved.
Chapter "Chapter 17" saved.
